# Review

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
sys.path.append(os.path.abspath(r"C:\ThacSi\HoachDinhCNTT\HDCNTT"))  # Thêm đường dẫn cha của thư mục wrangle
import utils, plot_help

from functools import reduce



%matplotlib inline

In [2]:
#modify read_limit and chunk_size parameters based on memory capacity
df = utils.chunk_loader('../data/review.json', read_limit=-1)

In [3]:
#extract year from datetime series
#make as type string for easier manipulation
df['year'] = df.date.dt.year.astype('str')
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,year
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,2018
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18,2012
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30,2014
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,2015
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,2017


## Group by business ID + Year and take the mean of reactions to the business

In [4]:
# Chỉ giữ lại các cột số
numeric_cols = ['stars', 'useful', 'funny', 'cool']

# Group theo business_id và year, rồi tính trung bình các cột số
review_year = df.groupby(['business_id', 'year'], as_index=False)[numeric_cols].mean()

# Sắp xếp kết quả nếu muốn
review_year = review_year.sort_values(by=['business_id', 'year'])

# Xem kết quả
review_year.head()


,business_id,year,stars,useful,funny,cool
0,---kPU91CF4Lq2-WlRu9Lw,2020,4.583333,1.000000,0.000000,0.750000
1,---kPU91CF4Lq2-WlRu9Lw,2021,4.363636,0.363636,0.090909,0.363636
2,---kPU91CF4Lq2-WlRu9Lw,2022,5.000000,0.000000,0.000000,0.000000
3,--0iUa4sNDFiZFrAdIWhZQ,2009,4.000000,5.000000,0.000000,1.000000
4,--0iUa4sNDFiZFrAdIWhZQ,2010,2.000000,1.000000,0.500000,0.000000


In [5]:
print(df.dtypes)


review_id              object
user_id                object
business_id            object
stars                   int64
useful                  int64
funny                   int64
cool                    int64
text                   object
date           datetime64[ns]
year                   object
dtype: object


## It is said that a business is successful if it can survive the test of time. We can use the review data to infer how a business is doing over the years. 
## For example we can measure the average change in star rating over the years. In the abscense of data in the year prior, assume the rating is unchanged since the last time it was recorded.

In [6]:
review_year.pivot(index='business_id', columns='year', values='stars').head()


year,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
business_id,,,,,,,,,,,,,,,,,,
---kPU91CF4Lq2-WlRu9Lw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.583333,4.363636,5.0
--0iUa4sNDFiZFrAdIWhZQ,NaN,NaN,NaN,NaN,4.0,2.0,NaN,4.0,4.0,3.5,3.0,1.0,1.0,NaN,NaN,NaN,5.000000,NaN
--30_8IhuyMHbSOcNWd6DQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,3.0,NaN,1.0,5.0,3.0,4.0,5.0,NaN,NaN,NaN
--7PUidqRWpRSpXebiyxTg,NaN,NaN,NaN,NaN,NaN,4.0,3.0,3.0,2.0,1.0,2.0,1.0,1.0,NaN,1.0,NaN,NaN,NaN
--7jw19RH9JKXgFohspgQw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,3.0,5.0,3.0,4.0,NaN,NaN,NaN


In [7]:
def feature_year_change(df_year, index, columns, values):
    
    #pivot table to evaluate change in stars
    df_year_feature = df_year.pivot(index=index, 
                                      columns=columns, 
                                      values=values)
    
    #convert to list of lists
    df_feature_list = df_year_feature.values

    df_no_nan = []

    #drop null values
    for array in df_feature_list:
        df_no_nan.append([x for x in array if pd.isna(x) != True])   

    df_no_nan = np.array([np.array(x) for x in df_no_nan])
    
    #save on memory
    del df_feature_list

    #get change in feature
    diffs = []
    for x in df_no_nan:
        if len(x)>2:
            diffs.append(np.diff(x))
        else:
            diffs.append(0)
    
    #again save memory
    del df_no_nan
    
    #get average change in feature value
    feature_change = []
    for x in diffs:
        feature_change.append(np.mean(x))
        
    #return as pandas object
    return pd.DataFrame.from_dict({'index': df_year_feature.index.tolist(),
                                   values: feature_change})


In [11]:
def feature_year_change(df_year, index, columns, values):
    # pivot để tạo bảng doanh nghiệp x năm
    df_year_feature = df_year.pivot(index=index, columns=columns, values=values)

    # danh sách các vector theo từng hàng (doanh nghiệp)
    df_feature_list = df_year_feature.values.tolist()

    # loại bỏ nan
    df_no_nan = [np.array([v for v in x if not pd.isna(v)]) for x in df_feature_list]

    # tính diff (chênh lệch giữa các năm)
    diffs = [np.diff(x) if len(x) > 1 else [0] for x in df_no_nan]

    # tính trung bình thay đổi
    feature_change = [np.mean(x) if len(x) > 0 else 0 for x in diffs]

    return pd.DataFrame.from_dict({
        'index': df_year_feature.index.tolist(),
        'values': feature_change
    })

In [12]:
feat_year_list = []

for feature in ['cool', 'funny', 'stars', 'useful']:
    feat_year_list.append(feature_year_change(review_year, 'business_id','year', feature))

In [13]:
feat_year_list[0].head()

,index,values
0,---kPU91CF4Lq2-WlRu9Lw,-0.375
1,--0iUa4sNDFiZFrAdIWhZQ,-0.125
2,--30_8IhuyMHbSOcNWd6DQ,0.000
3,--7PUidqRWpRSpXebiyxTg,0.000
4,--7jw19RH9JKXgFohspgQw,0.000


In [14]:
feat_year_list = []

for feature in ['cool', 'funny', 'stars', 'useful']:
    df_feat = feature_year_change(review_year, 'business_id', 'year', feature)
    df_feat.columns = ['index', feature]  # rename cột cho thống nhất
    feat_year_list.append(df_feat)

In [15]:
from functools import reduce

df_year_change = reduce(lambda left, right: pd.merge(left, right, on='index', how='outer'), feat_year_list)

# Đổi tên các cột
df_year_change = df_year_change.rename(columns={
    'index': 'business_id',
    'cool': 'cool_change',
    'funny': 'funny_change',
    'stars': 'stars_change',
    'useful': 'useful_change'
})


df_year_change.head()

,business_id,cool_change,funny_change,stars_change,useful_change
0,---kPU91CF4Lq2-WlRu9Lw,-0.375,0.000000,0.208333,-0.500
1,--0iUa4sNDFiZFrAdIWhZQ,-0.125,0.000000,0.125000,-0.625
2,--30_8IhuyMHbSOcNWd6DQ,0.000,-0.166667,0.000000,0.000
3,--7PUidqRWpRSpXebiyxTg,0.000,0.000000,-0.375000,-0.250
4,--7jw19RH9JKXgFohspgQw,0.000,0.000000,-0.200000,-0.800


In [17]:
df_year_change.to_csv('../data/cleaned/review_year_change.csv')